In [1]:
import numpy as np
import pandas as pd
import json
import datetime
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import uuid
import os
from os import path
from glob import glob
import json
import re
import pickle
import argparse

import numpy as np
from textwrap import wrap
from scipy import ndimage, misc

#import config
#from type import RecipeContainer, DataContainer
#from utils import url_to_filename

# import matplotlib using agg backend
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import elasticsearch
import boto3
import requests
import random
import names
import time

In [2]:
recipes_df = pd.read_json('recipes/bulk_openrecipes.json', orient='rows', lines=True)
recipes_df.head()
i = 0 
unique_ing_words = set()
for index, row in recipes_df.iterrows():
    i += 1
    #print(row['ingredients'])
    row_data = row['ingredients']
    if(isinstance(row_data, str)):
        unique_ing_words.update(row_data.lower().split())

#row['Food Name'].str.lower().str.split().apply(unique_words.update)
print(unique_ing_words)

{'combination)', 'everything', 'from', 'sharp', 'segment', 'pinto', 'mix)', 'scallions),', 'cottage', 'filling.', 'plenty', 'small)', 'apricots', 'silver', 'brush', 'scallions', 'boil.', 'shelf', 'shallots*', 'pickles', 'pieces,', 'yolks', 'hock,', 'even', 'tablespoon', 'booster,', 'club', '(hs', 'fluff', 'skillet,', 'peppered', 'san', 'crumbles', '(measure,', '115', 'containers', 'garlic', 'candy', 'mexican', 'tubes', 'mascarpone', 'angel', 'chops:', 'm&amp;ms,', 'prefer)', 'buttered,', 'start', 'gallo:', 'seeds)', 'stick,', 'parm,', 'squash,', 'natural', 'do)', 'half,', '(varied', 'penne)', 'cupcakes', 'by', 'amaranth', 'wedge', 'turkish', 'cracker', 'shaving', 'shiitake', 'artificial', 'louisiana', 'were', 'swiss', 'short', 'dutch-cocoa', 'zucchini)', '(or,', 'tots', 'rings', 'nectar)', 'oil:', '1/6th-inch', "file'", 'dino', 'get', 'storebought', '10)', 'variety)', 'pork,', '70f', 'seedless', 'washed,', 'ears', 'ripe,', 'powder/granulates', 'crunchy,', 'nervous', 'wheat,', 'root', '

In [3]:
zip_codes_df = pd.read_csv('zip/zip_codes.csv')
print(zip_codes_df.shape)
zip_codes_df.head()

(100, 3)


,ZIP_CODE,CITY,INCOME
0,79936,El Paso TX,42857
1,90011,Los Angeles CA,23851
2,60629,Chicago IL,40279
3,90650,Norwalk CA,46012
4,90201,Bell Gardens CA,30029


In [5]:
NUM_USERS  = 500

_occupations = ['butcher', 'baker', 'deli', 'pastry', 'chef', 'sue chef', 'candy maker']
_occupations_len = len(_occupations)
_languages = ['en', 'de', 'it', 'in', 'gu', 'ka', 'el']
_languages_len = len(_languages)
_food_items = unique_ing_words

def gen_users(num_users):
    _users = []
    _zip_len = zip_codes_df.shape[0]
    
    for i in range(num_users):
        _age        = int(np.random.normal(40, 4))
        _gender_idx = np.random.randint(0, 2)
        _loc_idx    = np.random.randint(0, _zip_len)
        _occupation_user = np.random.randint(0, _occupations_len)
        _languages_selected = np.random.randint(0, _languages_len)
        
        if _gender_idx == 0:
            _gender = 'MALE'
            _name = names.get_full_name(gender='male')
        else:
            _gender = 'FEMALE'
            _name = names.get_full_name(gender='female')
            
        _this_user = [str(i), _age, _gender, 
                      str(zip_codes_df.iloc[_loc_idx][0]), 
                      zip_codes_df.iloc[_loc_idx][2],
                      _occupations[_occupation_user],
                      _languages[_languages_selected],
                      ', '.join(random.sample(_food_items, 50)),
                      _name]
        
        _users.append(_this_user)
        
    _df = pd.DataFrame(_users, columns=['USER_ID', 'AGE', 'GENDER', 'LOCATION', 'SALARY', 'OCCUPATION', 'LANGUAGE', 'FAV', 'NAME'])
    return _df

users_df = gen_users(NUM_USERS)
print(users_df.shape)

users_df.to_csv('users/food_ind_generated_users.csv', index=False)

(500, 9)


In [6]:
users_df.head()

,USER_ID,AGE,GENDER,LOCATION,SALARY,OCCUPATION,LANGUAGE,FAV,NAME
0,0,40,MALE,78577,24216,chef,gu,"crumble, trini, open, splash, heat,, tenderize...",Clifton Rogers
1,1,42,MALE,30349,39141,chef,in,"cherry, kind, flakes),, tartar, (24, was, (the...",Darren Price
2,2,33,MALE,79912,48627,butcher,de,"fine),, bigger, panko, 6.5, (before, my, bagel...",Marcus Lake
3,3,48,FEMALE,91331,39225,baker,in,"try, while, diced, defrosted, peppers,, bunche...",Cinda Manos
4,4,37,FEMALE,10452,20606,deli,ka,"1/6-inch, 300, romanesco, (240g), blends., in....",Ida Gregory


In [ ]:
#Load into ElasticSearch
